In [3]:
#Importing Libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [5]:
# Creating Path
path = r'C:\Users\henry\OneDrive\042025Instacart Basket Analysis'

In [7]:
# Importing customer data
customers = pd.read_csv(os.path.join(path, '02 DATA', 'Original Data', 'customers.csv'))

In [9]:
# Checking customer data
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [13]:
customers.shape

(206209, 10)

In [15]:
customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [17]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


Wrangle the data so that it follows consistent logic

In [20]:
# Rename the variables to bring similar format

# Rename First Name as first_name
customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [22]:
# Rename Surnam as last_name
customers.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

In [24]:
# Rename Gender as gender
customers.rename(columns = {'Gender' : 'gender'}, inplace = True)

In [26]:
# Rename STATE as state
customers.rename(columns = {'STATE' : 'state'}, inplace = True)

In [28]:
# Rename Age as age
customers.rename(columns = {'Age' : 'age'}, inplace = True)

In [30]:
# Rename fam_status as marital_status
customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [32]:
# Checking Output
customers.columns

Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'n_dependants', 'marital_status', 'income'],
      dtype='object')

In [34]:
customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Complete the fundamental data quality and consistency checks

In [37]:
# finding missing values
missing_values = customers.isnull().sum()

In [39]:
missing_values

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
n_dependants          0
marital_status        0
income                0
dtype: int64

In [41]:
# first names hidden due to probale data protection issues

In [43]:
# Checking duplicates
df_dups = customers[customers.duplicated()]

In [45]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income


In [47]:
# no duplicates found

In [49]:
# Checking mixed-type
customers.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [51]:
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


In [53]:
# First name has a mixed data type
customers['first_name'] = customers['first_name'].astype('str')

In [55]:
customers.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [57]:
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

In [59]:
# All columns have right data type

Combine your customer data with the rest of your prepared Instacart data

In [62]:
# Importing prepared Instacart data
ords_prods_merge =pd.read_pickle(os.path.join(path, '02 DATA', 'Prepared Data', 'Aggregated_ords_prods_merge.pkl'))

In [63]:
# combining customer with ords_prods_merge
ords_prods_customers_merge = pd.merge(ords_prods_merge, customers, on='user_id', how='inner')

In [64]:
# Exporting new dataframe as a pickle file
ords_prods_customers_merge.to_pickle(os.path.join(path, '02 DATA','Prepared Data', 'orders_products_customers_aggregated.pkl'))